In [1]:
import json
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

In [2]:
# Load the data, create the one-hot labels, and prepare the K-Fold Cross Validation
CWD = os.getcwd()
string = os.path.join(CWD, 'data/put.mat')
all_data = loadmat(string) 
data_steps = all_data['features']
data_steps_array = np.array(data_steps)
data_labels = all_data['labels']
data_labels_arr = np.array(data_labels)
data_labels_arr = np.reshape(data_labels_arr, (data_labels_arr.shape[1]))

# normalization of the data
normed = preprocessing.scale(data_steps_array)    

normed, data_labels_arr = shuffle(normed, data_labels_arr, random_state=47)

# K-Fold Cross validation preparation
n_split = 10 # 10-fold cross validation
train_index = []
test_index = []
for train_ind,test_ind in KFold(n_split, random_state=47).split(normed):
    train_index.append(train_ind)
    test_index.append(test_ind)

In [3]:
# train the Support Vector Machines for Frequency Features based on paper:
# "What am I touching? Learning to classify terrain via haptic sensing"

val_acc = {'{:1d}'.format(j):[] for j in range(n_split)}
for i in range(n_split):
    svclassifier = SVC(kernel='rbf')
    x_train,x_test=normed[train_index[i]],normed[test_index[i]]
    y_train,y_test=data_labels_arr[train_index[i]],data_labels_arr[test_index[i]]
    svclassifier.fit(x_train, y_train)
    y_val = svclassifier.predict(x_test)
    tmp = classification_report(y_test,y_val)    
    val_acc['{:1d}'.format(i)] = float(tmp[388:400])

In [4]:
# The final Report
maxim = [np.max(val_acc['{:1d}'.format(i)]) for i in range(10)]

print('mean accuracy: ', np.mean(maxim)*100, ' std: ',  np.std(maxim)*100, ' min: ', np.min(maxim)*100, ' max: ', np.max(maxim)*100)

mean accuracy:  66.10000000000001  std:  1.6401219466856718  min:  64.0  max:  69.0
